In [1]:
!pip install FFmpeg
!pip install scikit-video

import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization,Flatten,Dropout

  Stored in directory: /root/.cache/pip/wheels/b6/68/c3/a05a35f647ba871e5572b9bbfc0b95fd1c6637a2219f959e7a
Successfully built FFmpeg
    100% |████████████████████████████████| 2.3MB 12.4MB/s 


Using TensorFlow backend.


Notations: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

Context
In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

$$\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \&gt; ,
\end{equation*}$$
where: \begin{equation}p^{\pi}(a{0},a{1},s{1},...,a{T},s{T})=p(a{0})\prod{t=1}^{T}\pi(a{t}|s{t})p(s{t+1}|s{t},a{t}) > . \end{equation}

We note the $Q$-function:

$$\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \&gt; .
\end{equation*}$$


Thus, the optimal Q function is:  𝑄∗(𝑠,𝑎)=max𝜋𝑄𝜋(𝑠,𝑎).


In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

The environment


Environment is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method act allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (game_over).

The method reset simply reinitializes the environment to a random state $s_0$.

The method draw displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

The Agent

The goal of the Agent is to interact with the Environment by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its reward $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e
    
    def get_epsilon(self):
        return self.epsilon

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass


Question 1: Explain the function act. Why is epsilon essential?

Answer 1:

It allows the agent to take an action given an observation of the current state of the environment. With several actions in place, we can randomly choose one.The problem is if we ever find a decent option, we’ll always choose that one in the future, even if there is a way better option available. To overcome this, we bring into the notion of exploration. Even after we’ve explored all the options and we know already the best one, we still choose a random action, that is to say, to keep exploring.

The standard way to get exploration is to introduce an additional term, epsilon. A value is generated randomly, if smaller than epsilon, a random action is chosen.  Otherwise we choose the optimal option available, which is  exploitation.

$$ a^{*} = argmax_{a} Q^{\pi} (s , a) $$
So, epsilon manage the tradeoff between exploration and exploitation thus :

During the exploration phase, the agent uses an epsilon greedy policy. Given a value for epsilon, the agent has a probability of epsilon to choose a random action. Otherwise, the agent will choose the action that gives the best reward. This policy allows the agent to explore new possibilities than the one it already considers as the best, wich allows to discover new strategies.

During the exploitation phase, we consider that the agent has explored enough stategies, we stop the epsilon greedy policy and then the agent will always select the action with the best reward.

The Game

The Agent and the Environment work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

In [5]:
epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0

    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()

TypeError: ignored

The game, eat cheese

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right.

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state


The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=10 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

Question 2 Explain the use of the arrays position and board.

Answer 2

Position is the array that contains the position of the rat on the board at a given time. The array is updated for each call of the function act by setting all the elements to zero except the position of the rat that is set to 1 and the boarders to -1. It records the position of the mouse and gives information about its movement.

Board is the array that contains information of the rewards on the board on which the rat moves around. Each cell can be neutral, reward(cheese) and penalty(poisonous cells) ,represented respectively by 0, 0.5 and -1. The distribution of the reward (0.5 and -1) follows a binomial law that is defined in the reset function.

The environment sends to the rat a partial view of position and  board (visibility of 2 cells from its current position : self.x and self.y)

Random Agent

Question 3 Implement a random Agent (only learned_act needs to be implemented):

In [0]:
import random as rd
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return rd.randint(0,3)


Question 4 Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        # At each epoch, we reset the game and return to the initial state
        state = env.reset()
        # We assume that the games will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent takes an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if game over or not
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [11]:

# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 11.5/8.0. Average score (3.5)
Win/lose count 10.5/11.0. Average score (1.5)
Win/lose count 7.0/15.0. Average score (-1.6666666666666667)
Win/lose count 12.0/16.0. Average score (-2.25)
Win/lose count 12.0/10.0. Average score (-1.4)
Win/lose count 8.5/7.0. Average score (-0.9166666666666666)
Win/lose count 10.5/11.0. Average score (-0.8571428571428571)
Win/lose count 15.0/8.0. Average score (0.125)
Win/lose count 12.5/22.0. Average score (-0.9444444444444444)
Win/lose count 8.5/12.0. Average score (-1.2)
Final score: -1.2


DQN

Let us assume here that $T=\infty$.

Question 5 Let $\pi$ be a policy, show that:

$$\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}$$
Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds:

$$\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}$$
Finally, deduce that a plausible objective is:

$$\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}$$




Answer 5 The Q-function is defined such that:

The action-value function can be written as 

Thus \begin{equation} Q^\pi(s,a)=E{p^{\pi}}[\sum{t\leq T}\gamma^{t}r(s{t},a{t})|s{0}=s,a{0}=a] > . \end{equation}

By using the Markov property and defining the function as a Bellman equation we have that:

$$\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}$$
One assumes that the optimal policy $\pi^*$ exists, the optimal Q function is defined as:

$$\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \&gt; 
\end{equation*}$$$$\begin{equation*}
Q^*(s,a)= \max_{\pi} E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')] \&gt; 
\end{equation*}$$$$\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma Q^{\pi^*}(s',a')] \&gt; 
\end{equation*}$$$$\begin{equation*}
Q^*(s,a)= E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}$$
Finally, a plausible objective is to minimize the loss function defined as:

$$\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}( ( Q^* (s,a,\theta ) - Q(s,a,\theta)) ^2) 
\end{equation*}$$$$\begin{equation*}
\mathcal{L}(\theta) =  E_{s' \sim \pi^*(.|s,a)} ( ( r(s,a)+\gamma\max_{a'}Q^{*}(s',a', \theta) - Q(s,a,\theta)) ^2) 
\end{equation*}$$$$\begin{equation*}
\mathcal{L}(\theta) = E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}$$



The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

Obtain the new state $s_{t+1}$ from the environment $p$;

Store $(s_t,a_t,s_{t+1})$;

Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.



Question 6 Implement the class Memory that stores moves (in a replay buffer) via remember and provides a random_access to these. Specify a maximum memory size to avoid side effects. You can for example use a list() and set by default max_memory=100.

In [6]:

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = deque(maxlen=self.max_memory)        

    def remember(self, m):
        # move m = list(s_t, s_(t+1), a_t, r_t, game_over)
        self.memory.append(m)

    def random_access(self):
        idx = rd.randint(0,len(self.memory)-1)
        return self.memory[idx]

The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))


Question 7 Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
    

    def learned_act(self, s):
        
        s = np.reshape(s,[1,5,5,self.n_state])
        # Predict the reward value based on the given state
        act_values = self.model.predict(s)
        return np.argmax(act_values[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        
        s_ = np.reshape(s_,[1,5,5,self.n_state])
        n_s_ = np.reshape(n_s_,[1,5,5,self.n_state])
        
        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        target_q = np.zeros((self.batch_size, 4))

        for i in range(self.batch_size):
            target = r_
            minibatch = self.memory.random_access()
            
            s_= minibatch[0]
            n_s_= minibatch[1]
            a_ = minibatch[2]
            r_= minibatch[3]
            game_over_= minibatch[4]
            
            input_states[i]= s_
            
            if game_over_:
                target= r_

            else:               
                target = (r_ + self.discount * np.amax(self.model.predict(n_s_)[0]))
                
            target_f = self.model.predict(s_)
            target_f[0][a_] = target
            self.model.fit(s_, target_f, epochs=1, verbose=0)
            
            target_q[i]=target_f
            
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            

class DQN_FC(DQN):
    def __init__(self, lr=0.1,  *args, **kwargs):
        super(DQN_FC, self).__init__(*args,**kwargs)
        
        # NN Model
        state_size = 50
        
        model = Sequential()
        model.add(Reshape((state_size,), input_shape=(5,5,self.n_state)))
        model.add(Dense(256, input_dim=state_size, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(4, activation='linear'))
        #model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        model.compile(loss='mse', optimizer=Adam(lr=lr))
        self.model = model


In [22]:
from collections import deque

env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(lr=0.1, grid_size=size, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train0.mp4'))

Instructions for updating:
Use tf.cast instead.
Epoch 000/010 | Loss 38.5756 | Win/lose count 3.0/10.0 (-7.0)
Epoch 001/010 | Loss 11.2830 | Win/lose count 5.5/3.0 (2.5)
Epoch 002/010 | Loss 0.0041 | Win/lose count 3.0/10.0 (-7.0)
Epoch 003/010 | Loss 0.2201 | Win/lose count 7.5/11.0 (-3.5)
Epoch 004/010 | Loss 0.1367 | Win/lose count 7.0/10.0 (-3.0)
Epoch 005/010 | Loss 0.5513 | Win/lose count 9.0/11.0 (-2.0)


KeyboardInterrupt: ignored

Question 8 Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [25]:
class DQN_CNN(DQN):
    def __init__(self, lr=0.1, *args,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        filters = 32
        kernel_size = (3,3)
        
        model = Sequential()
        model.add(Conv2D(filters, kernel_size, input_shape=(5,5,self.n_state), activation='relu'))
        model.add(Conv2D(filters, kernel_size, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Flatten())
        model.add(Dense(4, activation='relu'))
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(grid_size=size, lr=0.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

Epoch 000/010 | Loss 0.0003 | Win/lose count 3.0/4.0 (-1.0)
Epoch 001/010 | Loss 0.0034 | Win/lose count 2.5/3.0 (-0.5)
Epoch 002/010 | Loss 0.0103 | Win/lose count 5.5/2.0 (3.5)
Epoch 003/010 | Loss 0.0107 | Win/lose count 1.0/1.0 (0.0)
Epoch 004/010 | Loss 0.0088 | Win/lose count 3.5/3.0 (0.5)


KeyboardInterrupt: ignored

In [0]:
HTML(display_videos('cnn_train2.mp4'))

Question 9 Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [26]:
# the performances are shown in the previous exercise, here we test different temperature with 5 epochs
temp = [0.1,0.3, 0.5, 1]
for temp1 in temp:
    print("\n temperature : {0}".format(temp1)  )   
    env = Environment(grid_size=size, max_time=T, temperature=temp1)
    agent = DQN_CNN(lr=0.1, grid_size= size, epsilon = 0.1, memory_size=2000, batch_size = 32)
    train(agent,env,5,prefix='cnn_train')
    


 temperature : 0.1
Epoch 000/005 | Loss 0.0000 | Win/lose count 1.0/2.0 (-1.0)
Epoch 001/005 | Loss 0.0000 | Win/lose count 0/1.0 (-1.0)
Epoch 002/005 | Loss 0.0078 | Win/lose count 1.0/2.0 (-1.0)
Epoch 003/005 | Loss 0.0000 | Win/lose count 0.5/0 (0.5)
Epoch 004/005 | Loss 0.0000 | Win/lose count 1.5/2.0 (-0.5)
Final score: -0.6

 temperature : 0.3
Epoch 000/005 | Loss 0.0001 | Win/lose count 1.5/0 (1.5)
Epoch 001/005 | Loss 0.0024 | Win/lose count 5.0/2.0 (3.0)
Epoch 002/005 | Loss 0.0038 | Win/lose count 1.0/3.0 (-2.0)
Epoch 003/005 | Loss 0.0001 | Win/lose count 4.0/7.0 (-3.0)
Epoch 004/005 | Loss 0.0132 | Win/lose count 3.5/4.0 (-0.5)
Final score: -0.2

 temperature : 0.5
Epoch 000/005 | Loss 0.0975 | Win/lose count 2.5/2.0 (0.5)
Epoch 001/005 | Loss 0.0265 | Win/lose count 8.0/4.0 (4.0)
Epoch 002/005 | Loss 0.0072 | Win/lose count 3.5/3.0 (0.5)
Epoch 003/005 | Loss 0.0095 | Win/lose count 6.0/3.0 (3.0)
Epoch 004/005 | Loss 0.0123 | Win/lose count 6.0/6.0 (0.0)
Final score: 1.6



We can notice that the results given by the CNN are better than by the FC model


By increasing the temperature,we improve the final score, butby doing so we increase the probability of having cheese and lower the probility of poisonious cells (when temperature is 1, there's only cheese)

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:

Incorporating a decreasing  𝜖 -greedy exploration. You can use the method set_epsilon
Append via the environment a new state that describes if a cell has been visited or not

Question 10 Design a new train_explore function and environment class EnvironmentExploring to tackle the issue of exploration.

In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of games wons
    score = 0
    loss = 0

    
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            
            # Epsilon-greedy 
            agent.set_epsilon(np.float(agent.get_epsilon())*0.99)
            
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
            
        # Save as a mp4       
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
    print('Final score: '+str(score/epoch))
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        
        self.board = np.zeros([grid_size,grid_size])
        self.position = np.zeros([grid_size,grid_size])
        self.malus_position = np.zeros([grid_size,grid_size])
        
        # coordinate of the rat
        self.x = 0
        self.y = 1
        
        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))
        
    def get_t(self):
        return self.t
    
    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b
             
        
    def act(self, action, train=True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        #directions of movement
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        
        self.t = self.t + 1
        
        ## In Environment exploring:
        # You will have to change n_state to 3 because you will use one more layer!
        reward = 0
        if train:
            reward -= self.malus_position[self.x, self.y]

        self.malus_position[self.x, self.y] = 0.01
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time

        # 3 "feature" states instead of 2
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        
        return state, reward, game_over
    
    
    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
       
        self.malus_position[self.x, self.y] = 0

        self.t = 0

        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        
        return state

In [0]:
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(lr=.1, grid_size=size,  epsilon = 0.1, memory_size=16000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/020 | Loss 0.0041 | Win/lose count 3.0/5.769999999999962 (-2.7699999999999623)
Epoch 001/020 | Loss 0.0041 | Win/lose count 3.46/4.839999999999964 (-1.3799999999999644)
Epoch 002/020 | Loss 0.0017 | Win/lose count 4.98/3.8699999999999823 (1.110000000000018)
Epoch 003/020 | Loss 0.0015 | Win/lose count 4.940000000000001/4.819999999999983 (0.12000000000001787)
Epoch 004/020 | Loss 0.0112 | Win/lose count 7.370000000000002/2.789999999999984 (4.580000000000018)
Epoch 005/020 | Loss 0.0075 | Win/lose count 10.350000000000003/2.739999999999985 (7.610000000000018)
Epoch 006/020 | Loss 0.0505 | Win/lose count 6.8900000000000015/2.869999999999985 (4.0200000000000164)
Epoch 007/020 | Loss 0.0771 | Win/lose count 4.410000000000001/4.9099999999999815 (-0.49999999999998046)
Epoch 008/020 | Loss 0.0254 | Win/lose count 3.920000000000001/6.919999999999961 (-2.99999999999996)
Epoch 009/020 | Loss 0.0011 | Win/lose count 3.4400000000000004/2.9399999999999804 (0.50000000000002)
Epoch 010/020 |

In [0]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore0.mp4'))

Win/lose count 21.56999999999999/1.5600000000000012. Average score (20.009999999999987)
Win/lose count 5.8900000000000015/1.8900000000000015. Average score (12.004999999999994)
Win/lose count 7.350000000000002/2.859999999999982. Average score (9.500000000000002)
Win/lose count 5.3900000000000015/1.8900000000000015. Average score (8.000000000000002)
Win/lose count 0.98/1.9900000000000015. Average score (6.198)
Win/lose count 11.270000000000003/1.7700000000000014. Average score (6.748333333333334)
Win/lose count 13.730000000000004/2.7299999999999853. Average score (7.355714285714289)
Win/lose count 8.830000000000002/1.8200000000000014. Average score (7.312500000000003)
Win/lose count 5.880000000000002/1.8800000000000014. Average score (6.944444444444446)
Win/lose count 0.49/2.0000000000000013. Average score (6.099000000000002)
Win/lose count 3.4300000000000006/2.939999999999981. Average score (5.589090909090913)
Win/lose count 1.47/1.9800000000000015. Average score (5.080833333333337)
Wi